In [1]:
import torch

if torch.cuda.is_available():
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available.")

GPU is available: NVIDIA A10G


In [2]:
from huggingface_hub import login
import getpass

# Prompt the user to enter the Hugging Face token securely
token = getpass.getpass("Enter your Hugging Face token: ")

# Login to Hugging Face using the token
login(token=token)

Enter your Hugging Face token:  ········


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# load the model
# model_id = "meta-llama/Llama-2-7b-chat-hf"
model_id = "meta-llama/Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [4]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype=torch.float16)

In [6]:
INSTRUCTION = \
"""
You are an assistant trained to detect if a question is asking about a future event. Here are some examples:

Example 1:
Question: "When will the next meeting happen?"
Answer: Yes

Example 2:
Question: "What happened during the last meeting?"
Answer: No

Example 3:
Question: "What will be covered in the next conference?"
Answer: Yes

Now analyze the following question and decide if it refers to a future event:
Question:
"""

In [23]:
# torch.cuda.empty_cache()
# read in the doc
import pandas as pd
csv_path = '../data/annotated/QA_pairs_2.csv'
df = pd.read_csv(csv_path)
questions = []
# Iterate through the rows in the dataframe
for index, row in df.iterrows():
    questions.append(row['Question'])

In [24]:
len(questions)

3938

In [25]:
# import pandas as pd
from tqdm import tqdm
import datetime
results = []
for question in tqdm(questions):
# formatted_input = INSTRUCTION + " " + "When did Pittsburgh welcome a series of major film and television productions?" + "\n\n" + "Is this question about a future event? Please answer 'Yes' or 'No'."
    formatted_input = INSTRUCTION + " " + f"{question}" + "\n\n" + "Is this question about a future event? Please answer 'Yes' or 'No'."
    
    tokenized_prompt = tokenizer(formatted_input, return_tensors="pt").to(model.device)
        
    # Generate the response
    messages = [
        {"role": "user", "content": formatted_input},
    ]
    with torch.no_grad():
        result = pipe(messages, max_new_tokens=512)
    
    results.append(result[0]['generated_text'][1]['content'])

100%|███████████████████████████████████████████████████████████| 3938/3938 [09:21<00:00,  7.01it/s]


In [19]:
df['is_future_event'] = results

In [22]:
df.to_csv('../data/annotated/QA_pairs_2_with_future_event.csv')